# Quality Control of Cadastral Map Data Export

The goal of this project was to test the quality of the data exports of cadastral maps from a geoinformatics software MapSoft. Specifically, this program controls the space calculations of geocomponents and land parcels within greater geocomplexes. The data extracted from the xml files is in line with the data requirements of the control project and/or the systematization of the cadastre of Republic of Srpska (Bosnia & Herzegovina). All labels are in Serbian. 

In [1]:
import re
import pandas as pd

### Import XML File

In [6]:
# this is one xml file example

ulazni_fajl = "TRNOVICA_2500.xml"

### Data Extraction

Creating a new class "Geokompleks" to organize the data extracted from the xml file. A new class object is instaniated for every geocomplex and its associated land parcels. The attributes represent relevant data categories, such as number, type and area. The class method "kao_recnik" is used to organize the data points from each class object into a dictionary which is later passed into a dataframe. 

In [23]:
class Geokompleks:
    def __init__(self, brojac, kategorija, broj_parcele, podbroj_parcele, broj_poligona, nacin_koriscenja, povrsina, broj_linija):
        self.brojac = brojac
        self.kategorija = kategorija
        self.broj_parcele = broj_parcele
        self.podbroj_parcele = podbroj_parcele
        self.broj_poligona = broj_poligona
        self.nacin_koriscenja = nacin_koriscenja
        self.povrsina = povrsina
        self.broj_linija = broj_linija
        
    def kao_recnik(self):
        return {
            "Brojac" : self.brojac,
            "Kategorija" : self.kategorija, 
            "Broj Parcele" : self.broj_parcele,
            "Podbroj Parcele" : self.podbroj_parcele,
            "Broj Poligona" : self.broj_poligona,
            "Nacin Koriscenja" : self.nacin_koriscenja, 
            "Povrsina" : self.povrsina, 
            "Broj linija" : self.broj_linija
        }
    

Extracts data using the readline method and regular expression in order to identify the needed data points for every parcel/geocomplex. Stores the data points into variables which match the Geokompleks class attributes. Identifies each new parcel so that a class object is instatiated and added to a list. 

In [24]:
lista_redova = []

with open(ulazni_fajl, 'r') as ulaz:
    
    
    brojac_geokompleksa = 0
    brojac_poligona = -1
    brojac_linija = 0
    
    
    while True:
        line = ulaz.readline()
        if "<geokompleks>" in line:
            brojac_geokompleksa += 1
            print(f"{brojac_geokompleksa} geokompleks")
            
            while True:
                geokompleks_line = ulaz.readline()    
                if "<poligon>" in geokompleks_line:
                    broj_oznaka = 0
                    brojac_poligona += 1
                    print(f"{brojac_poligona} poligon")
                    brojac = brojac_poligona
                    kategorija = "Parcela"
                    
                    while True:
                        poligon_line = ulaz.readline()  
                        
                        if "<linija>" in poligon_line: 
                            brojac_linija +=1
                            
                        if "<povrsina>" and "</povrsina>" in poligon_line:
                            povrsina_pattern = re.findall("<povrsina>(.+?)</povrsina>", poligon_line)
                            print(f"Povrsina: {float(povrsina_pattern[0])}")
                            povrsina = float(povrsina_pattern[0])
        
                        if brojac_poligona==0 and "<tipOpisa>1</tipOpisa>" in poligon_line:
                            while True:
                                druge_line = ulaz.readline()
                                if "<tekst>" and "</tekst>" in druge_line:
                                    broj_parcele_pattern = re.findall("<tekst>(.+?)</tekst>", druge_line)
                                    print(f"Broj parcele: {broj_parcele_pattern[0]}")
                                    if "/" in broj_parcele_pattern[0]: 
                                        glavni_broj_pattern = re.findall("(\d+)?\/", broj_parcele_pattern[0])
                                        broj_parcele = glavni_broj_pattern[0]
                                        podbroj_pattern = re.findall("\/(\d+)", broj_parcele_pattern[0])
                                        podbroj_parcele = podbroj_pattern[0]
                                    else:
                                        broj_parcele = broj_parcele_pattern[0]
                                        podbroj_parcele = 0
                                    broj_poligona = "0"
                                    print("Nacin koriscenja: 440")
                                    nacin_koriscenja = "440"
                                    break
                                    
                        if brojac_poligona>0:
                            brojac = brojac_poligona
                            kategorija = "Poligon"
                            if "<tipOpisa>3</tipOpisa>" in poligon_line:
                                while True:
                                    druge_line = ulaz.readline()
                                    if "<tekst>" and "</tekst>" in druge_line:
                                        broj_parcele_pattern = re.findall("<tekst>(.+?)</tekst>", druge_line)
                                        print(f"Broj poligona: {broj_parcele_pattern[0]}")
                                        broj_poligona = broj_parcele_pattern[0]
                                    if "<oznaka>" and "</oznaka>" in druge_line:
                                        broj_oznaka += 1
                                        nacin_koriscenja_pattern = re.findall("<oznaka>(.+?)</oznaka>", druge_line)
                                        print(f"Nacin koriscenja: {nacin_koriscenja_pattern[0]}")
                                        nacin_koriscenja = nacin_koriscenja_pattern[0]
                                        break
                                        
                            if broj_oznaka==0 and ("<oznaka>" and "</oznaka>" in poligon_line):
                                nacin_koriscenja_pattern = re.findall("<oznaka>(.+?)</oznaka>", poligon_line)
                                print(f"Nacin koriscenja: {nacin_koriscenja_pattern[0]}")
                                broj_oznaka += 1
                                broj = " "
                                nacin_koriscenja = nacin_koriscenja_pattern[0]                              
                        
                                    
                        if "</poligon>" in poligon_line:
                            print(f"Broj linija: {brojac_linija}")
                            broj_linija = brojac_linija
                            brojac_linija = 0
                            red_tabele = Geokompleks(brojac, kategorija, broj_parcele, podbroj_parcele, broj_poligona, nacin_koriscenja, povrsina, broj_linija)
                            lista_redova.append(red_tabele)
                            break
                            
                    
                if "</geokompleks>" in geokompleks_line:
                    brojac_poligona = -1
                    break
                    
                    
        if not line:
            break


1 geokompleks
0 poligon
Broj parcele: 299
Nacin koriscenja: 440
Povrsina: 2446.16
Broj linija: 13
1 poligon
Broj poligona: 2
Nacin koriscenja: 198
Povrsina: 789.92
Broj linija: 24
2 poligon
Broj poligona: 1
Nacin koriscenja: 214
Povrsina: 1656.24
Broj linija: 27
2 geokompleks
0 poligon
Broj parcele: 354
Nacin koriscenja: 440
Povrsina: 3513.76
Broj linija: 18
1 poligon
Broj poligona: 2
Nacin koriscenja: 27
Povrsina: 62.01
Broj linija: 4
2 poligon
Broj poligona: 1
Nacin koriscenja: 25
Povrsina: 61.82
Broj linija: 7
3 poligon
Broj poligona: 2
Nacin koriscenja: 199
Povrsina: 865.88
Broj linija: 8
4 poligon
Broj poligona: 1
Nacin koriscenja: 219
Povrsina: 2647.89
Broj linija: 12
3 geokompleks
0 poligon
Broj parcele: 176
Nacin koriscenja: 440
Povrsina: 5699.91
Broj linija: 15
1 poligon
Broj poligona: 2
Nacin koriscenja: 198
Povrsina: 969.93
Broj linija: 16
2 poligon
Broj poligona: 1
Nacin koriscenja: 215
Povrsina: 4729.97
Broj linija: 25
4 geokompleks
0 poligon
Broj parcele: 242
Nacin korisc

Povrsina: 2531.18
Broj linija: 12
72 geokompleks
0 poligon
Broj parcele: 485
Nacin koriscenja: 440
Povrsina: 1978.24
Broj linija: 7
1 poligon
Nacin koriscenja: 215
Povrsina: 1978.24
Broj linija: 7
73 geokompleks
0 poligon
Broj parcele: 484
Nacin koriscenja: 440
Povrsina: 2841.85
Broj linija: 19
1 poligon
Nacin koriscenja: 199
Povrsina: 2841.85
Broj linija: 19
74 geokompleks
0 poligon
Broj parcele: 483
Nacin koriscenja: 440
Povrsina: 3125.47
Broj linija: 5
1 poligon
Nacin koriscenja: 199
Povrsina: 3125.47
Broj linija: 5
75 geokompleks
0 poligon
Broj parcele: 482
Nacin koriscenja: 440
Povrsina: 23231.82
Broj linija: 14
1 poligon
Nacin koriscenja: 199
Povrsina: 23231.82
Broj linija: 14
76 geokompleks
0 poligon
Broj parcele: 481
Nacin koriscenja: 440
Povrsina: 2532.76
Broj linija: 8
1 poligon
Nacin koriscenja: 215
Povrsina: 2532.76
Broj linija: 8
77 geokompleks
0 poligon
Broj parcele: 480
Nacin koriscenja: 440
Povrsina: 1254.85
Broj linija: 6
1 poligon
Nacin koriscenja: 214
Povrsina: 1254.

Nacin koriscenja: 507
Povrsina: 81.02
Broj linija: 4
2 poligon
Broj poligona: 1
Nacin koriscenja: 507
Povrsina: 77.74
Broj linija: 4
3 poligon
Broj poligona: 2
Nacin koriscenja: 200
Povrsina: 198.91
Broj linija: 6
4 poligon
Broj poligona: 1
Nacin koriscenja: 215
Povrsina: 24755.15
Broj linija: 54
124 geokompleks
0 poligon
Broj parcele: 432
Nacin koriscenja: 440
Povrsina: 2936.12
Broj linija: 7
1 poligon
Broj poligona: 1
Nacin koriscenja: 215
Povrsina: 2699.1
Broj linija: 18
2 poligon
Broj poligona: 2
Nacin koriscenja: 199
Povrsina: 237.03
Broj linija: 15
125 geokompleks
0 poligon
Broj parcele: 431
Nacin koriscenja: 440
Povrsina: 2282.27
Broj linija: 8
1 poligon
Broj poligona: 2
Nacin koriscenja: 199
Povrsina: 167.39
Broj linija: 21
2 poligon
Broj poligona: 1
Nacin koriscenja: 215
Povrsina: 2114.87
Broj linija: 23
126 geokompleks
0 poligon
Broj parcele: 430
Nacin koriscenja: 440
Povrsina: 4450.67
Broj linija: 50
1 poligon
Nacin koriscenja: 215
Povrsina: 4450.67
Broj linija: 50
127 geoko

Povrsina: 17997.69
Broj linija: 31
1 poligon
Nacin koriscenja: 199
Povrsina: 17997.69
Broj linija: 31
210 geokompleks
0 poligon
Broj parcele: 342
Nacin koriscenja: 440
Povrsina: 2730.97
Broj linija: 51
1 poligon
Nacin koriscenja: 215
Povrsina: 2730.97
Broj linija: 51
211 geokompleks
0 poligon
Broj parcele: 341
Nacin koriscenja: 440
Povrsina: 15879.77
Broj linija: 58
1 poligon
Nacin koriscenja: 199
Povrsina: 15879.77
Broj linija: 58
212 geokompleks
0 poligon
Broj parcele: 340
Nacin koriscenja: 440
Povrsina: 8142.71
Broj linija: 14
1 poligon
Broj poligona: 2
Nacin koriscenja: 215
Povrsina: 513.77
Broj linija: 66
2 poligon
Broj poligona: 1
Nacin koriscenja: 199
Povrsina: 7628.94
Broj linija: 14
213 geokompleks
0 poligon
Broj parcele: 339
Nacin koriscenja: 440
Povrsina: 1715.37
Broj linija: 7
1 poligon
Nacin koriscenja: 215
Povrsina: 1715.37
Broj linija: 7
214 geokompleks
0 poligon
Broj parcele: 338
Nacin koriscenja: 440
Povrsina: 5103.58
Broj linija: 28
1 poligon
Nacin koriscenja: 215
Pov

Povrsina: 3215.4
Broj linija: 47
1 poligon
Nacin koriscenja: 198
Povrsina: 3215.4
Broj linija: 47
285 geokompleks
0 poligon
Broj parcele: 264
Nacin koriscenja: 440
Povrsina: 1643.77
Broj linija: 9
1 poligon
Nacin koriscenja: 215
Povrsina: 1643.77
Broj linija: 9
286 geokompleks
0 poligon
Broj parcele: 263
Nacin koriscenja: 440
Povrsina: 1791.53
Broj linija: 68
1 poligon
Nacin koriscenja: 215
Povrsina: 1791.53
Broj linija: 68
287 geokompleks
0 poligon
Broj parcele: 262
Nacin koriscenja: 440
Povrsina: 48503.79
Broj linija: 149
1 poligon
Broj poligona: 4
Nacin koriscenja: 215
Povrsina: 456.9
Broj linija: 82
2 poligon
Broj poligona: 3
Nacin koriscenja: 215
Povrsina: 767.03
Broj linija: 67
3 poligon
Broj poligona: 2
Nacin koriscenja: 215
Povrsina: 831.88
Broj linija: 57
4 poligon
Broj poligona: 1
Nacin koriscenja: 198
Povrsina: 46447.97
Broj linija: 214
288 geokompleks
0 poligon
Broj parcele: 261
Nacin koriscenja: 440
Povrsina: 5381.34
Broj linija: 10
1 poligon
Broj poligona: 2
Nacin korisce

Povrsina: 2002.56
Broj linija: 8
1 poligon
Nacin koriscenja: 215
Povrsina: 2002.56
Broj linija: 8
366 geokompleks
0 poligon
Broj parcele: 182
Nacin koriscenja: 440
Povrsina: 3932.71
Broj linija: 9
1 poligon
Broj poligona: 2
Nacin koriscenja: 198
Povrsina: 1173.44
Broj linija: 24
2 poligon
Broj poligona: 1
Nacin koriscenja: 214
Povrsina: 2759.27
Broj linija: 25
367 geokompleks
0 poligon
Broj parcele: 181
Nacin koriscenja: 440
Povrsina: 1922.17
Broj linija: 9
1 poligon
Broj poligona: 2
Nacin koriscenja: 27
Povrsina: 44.97
Broj linija: 4
2 poligon
Broj poligona: 3
Nacin koriscenja: 27
Povrsina: 41.56
Broj linija: 4
3 poligon
Broj poligona: 1
Nacin koriscenja: 25
Povrsina: 59.99
Broj linija: 4
368 geokompleks
0 poligon
Broj parcele: 180
Nacin koriscenja: 440
Povrsina: 2226.81
Broj linija: 10
1 poligon
Broj poligona: 2
Nacin koriscenja: 27
Povrsina: 49.99
Broj linija: 4
2 poligon
Broj poligona: 1
Nacin koriscenja: 25
Povrsina: 60.45
Broj linija: 4
369 geokompleks
0 poligon
Broj parcele: 179

Povrsina: 2701.24
Broj linija: 47
1 poligon
Broj poligona: 2
Nacin koriscenja: 27
Povrsina: 31.96
Broj linija: 4
2 poligon
Broj poligona: 1
Nacin koriscenja: 25
Povrsina: 60.47
Broj linija: 4
407 geokompleks
0 poligon
Broj parcele: 140
Nacin koriscenja: 440
Povrsina: 2713.24
Broj linija: 71
1 poligon
Broj poligona: 2
Nacin koriscenja: 27
Povrsina: 44.96
Broj linija: 4
2 poligon
Broj poligona: 1
Nacin koriscenja: 25
Povrsina: 78.03
Broj linija: 4
408 geokompleks
0 poligon
Broj parcele: 139
Nacin koriscenja: 440
Povrsina: 36018.53
Broj linija: 108
1 poligon
Nacin koriscenja: 214
Povrsina: 36018.53
Broj linija: 108
409 geokompleks
0 poligon
Broj parcele: 138
Nacin koriscenja: 440
Povrsina: 15231.29
Broj linija: 42
1 poligon
Nacin koriscenja: 214
Povrsina: 15231.29
Broj linija: 42
410 geokompleks
0 poligon
Broj parcele: 137
Nacin koriscenja: 440
Povrsina: 8290.68
Broj linija: 57
1 poligon
Nacin koriscenja: 214
Povrsina: 8290.68
Broj linija: 57
411 geokompleks
0 poligon
Broj parcele: 136
Na

Broj parcele: 68
Nacin koriscenja: 440
Povrsina: 6490.54
Broj linija: 30
1 poligon
Nacin koriscenja: 199
Povrsina: 6490.54
Broj linija: 30
478 geokompleks
0 poligon
Broj parcele: 67
Nacin koriscenja: 440
Povrsina: 5655.24
Broj linija: 40
1 poligon
Nacin koriscenja: 215
Povrsina: 5655.24
Broj linija: 40
479 geokompleks
0 poligon
Broj parcele: 66
Nacin koriscenja: 440
Povrsina: 1334.93
Broj linija: 39
1 poligon
Nacin koriscenja: 215
Povrsina: 1334.93
Broj linija: 39
480 geokompleks
0 poligon
Broj parcele: 65
Nacin koriscenja: 440
Povrsina: 1911.35
Broj linija: 30
1 poligon
Nacin koriscenja: 198
Povrsina: 1911.35
Broj linija: 30
481 geokompleks
0 poligon
Broj parcele: 64
Nacin koriscenja: 440
Povrsina: 29587.1
Broj linija: 153
1 poligon
Nacin koriscenja: 199
Povrsina: 29587.1
Broj linija: 153
482 geokompleks
0 poligon
Broj parcele: 63
Nacin koriscenja: 440
Povrsina: 11005.62
Broj linija: 116
1 poligon
Nacin koriscenja: 215
Povrsina: 11005.62
Broj linija: 116
483 geokompleks
0 poligon
Broj

### Convert Data Into a Table

Iterates through the list of class objects to convert them into dictionaries and pass them to a Dataframe.

In [25]:
tabela_df = pd.DataFrame([red_tabele.kao_recnik() for red_tabele in lista_redova])


brojac_lista = list(tabela_df["Brojac"])
povrsina_lista = list(tabela_df["Povrsina"])


brojac_lista_rev = brojac_lista[::-1]
povrsina_lista_rev = povrsina_lista[::-1]


sum = 0
control_total_rev = []
for brojac, povrsina in zip(brojac_lista_rev, povrsina_lista_rev):
    
    if brojac > 0: 
        sum += povrsina
        control_total_rev.append(0)

    else:
        control_total_rev.append(round(sum, 2))
        sum = 0

control_total = control_total_rev[::-1]

tabela_df["Kontrola Totala"] = control_total

tabela_df.loc[tabela_df["Brojac"]==0, "Razlika"] = tabela_df["Povrsina"] - tabela_df["Kontrola Totala"]
tabela_df.loc[tabela_df["Brojac"]!=0, "Razlika"] = 0

tabela_df.loc[abs(tabela_df["Razlika"])<1, "Greska"] = " "
tabela_df.loc[abs(tabela_df["Razlika"])>1, "Greska"] = "Greska"

tabela_df

,Brojac,Kategorija,Broj Parcele,Podbroj Parcele,Broj Poligona,Nacin Koriscenja,Povrsina,Broj linija,Kontrola Totala,Razlika,Greska
0,0,Parcela,299,0,0,440,2446.16,13,2446.16,0.00,
1,1,Poligon,299,0,2,198,789.92,24,0.00,0.00,
2,2,Poligon,299,0,1,214,1656.24,27,0.00,0.00,
3,0,Parcela,354,0,0,440,3513.76,18,3637.60,-123.84,Greska
4,1,Poligon,354,0,2,27,62.01,4,0.00,0.00,
...,...,...,...,...,...,...,...,...,...,...,...
1275,1,Poligon,3,0,0,199,5267.14,131,0.00,0.00,
1276,0,Parcela,2,0,0,440,4255.68,90,4255.68,0.00,
1277,1,Poligon,2,0,0,214,4255.68,90,0.00,0.00,
1278,0,Parcela,1,0,0,440,28389.52,48,28389.52,0.00,


### Table Export

In [26]:
tabela_df.to_csv('TRNOVICA_2500.csv')
tabela_df.to_excel('TRNOVICA_2500_1.xlsx')